Dispositivo	País	Franja Horaria	Compra (Target)
Móvil	España	Noche	Sí
Ordenador	México	Tarde	No
Tablet	España	Mañana	Sí
Móvil	Estados Unidos	Noche	No
Ordenador	México	Noche	Sí
Tablet	España	Tarde	No

In [29]:
import pandas as pd

In [30]:
df = pd.DataFrame({
    "Dispositivo": ["Movil", "Tablet", "PC", "Movil", "Tablet", "PC"],
    "Pais": ["ES", "ES", "ES", "US", "US", "US"],
    "Franja Horaria": ["Mañana", "Mañana", "Mañana", "Tarde", "Tarde", "Tarde"],
    "Compra":["Si", "No", "Si", "No", "Si", "No"]
})

In [31]:
df

,Dispositivo,Pais,Franja Horaria,Compra
0,Movil,ES,Mañana,Si
1,Tablet,ES,Mañana,No
2,PC,ES,Mañana,Si
3,Movil,US,Tarde,No
4,Tablet,US,Tarde,Si
5,PC,US,Tarde,No


In [32]:
df_one_hot = pd.get_dummies(df, columns=["Dispositivo", "Pais", "Franja Horaria"])
df_one_hot['Compra'] = df_one_hot['Compra'].map({'Si': 1, 'No': 0})


In [33]:
df_one_hot

,Compra,Dispositivo_Movil,Dispositivo_PC,Dispositivo_Tablet,Pais_ES,Pais_US,Franja Horaria_Mañana,Franja Horaria_Tarde
0,1,True,False,False,True,False,True,False
1,0,False,False,True,True,False,True,False
2,1,False,True,False,True,False,True,False
3,0,True,False,False,False,True,False,True
4,1,False,False,True,False,True,False,True
5,0,False,True,False,False,True,False,True


In [34]:
from sklearn.model_selection import train_test_split

# Variables independientes (X) y dependiente (y)
X = df_one_hot.drop('Compra', axis=1)
y = df_one_hot['Compra']

# Dividir el dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [47]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np

# Supongamos que X e y representan tu dataset de características y etiquetas
# Calcular los parámetros de escalado sobre el dataset completo
scaler = StandardScaler()
scaler.fit(X)  # Ajusta el escalador con el dataset completo

# Definir el modelo MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1, warm_start=True)

# Definir las clases del modelo (es importante para partial_fit)
classes = np.unique(y)

# Dividir el dataset en chunks y entrenar con partial_fit
chunk_size = 1000  # Define el tamaño del chunk

def get_chunks(X, y, chunk_size):
    
    id_random = np.random.permutation(len(X))
    X = X.iloc[id_random]
    y = y.iloc[id_random]

    return X, y

for i in range(0, len(X), chunk_size):
    
    # Obtener el chunk actual
    X_chunk, y_chunk = get_chunks(X_train, y_train, chunk_size)
    
    # Aplicar la normalización a cada chunk utilizando el escalador ajustado
    X_chunk = scaler.transform(X_chunk)
    
    # Entrenamiento incremental en el chunk actual
    model.partial_fit(X_chunk, y_chunk, classes=classes)


In [42]:

# Predicciones
y_pred = model.predict(X_test)


/home/djoroya/Documents/desafio/hackathon-caixabank-data-ai-report/.conda/lib/python3.13/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but MLPClassifier was fitted without feature names
  warnings.warn(


In [43]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Exactitud del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Exactitud del modelo:", accuracy)


Exactitud del modelo: 1.0


In [44]:

# Matriz de confusión
print("Matriz de Confusión:")
mc = confusion_matrix(y_test, y_pred)

print("Verdaderos positivos:", mc[0][0])
print("Falsos positivos:", mc[0][1])
print("Falsos negativos:", mc[1][0])
print("Verdaderos negativos:", mc[1][1])




Matriz de Confusión:
Verdaderos positivos: 1
Falsos positivos: 0
Falsos negativos: 0
Verdaderos negativos: 1


In [45]:

# Reporte de clasificación
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))


Reporte de Clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

